In [ ]:
import matplotlib
%matplotlib widget

import numpy as np
from split_dataset import SplitDataset
from pathlib import Path
import flammkuchen as fl
import matplotlib.pyplot as plt 
from fimpylab.core.lightsheet_experiment import LightsheetExperiment
from bouterin.plots.stimulus_log_plot import get_paint_function

from bouter.utilities import reliability 
from skimage.filters import threshold_otsu
import xarray as xr
from scipy.signal import detrend 

from motions.utilities import stim_vel_dir_dataframe, quantize_directions
import tifffile as tiff
from scipy.signal import argrelextrema
from scipy.signal import find_peaks

In [ ]:
master_path =  Path(r"Z:\Hagar\E0040\ablations\pre\v31")
fish_list = list(master_path.glob("*f*"))
thresh = 0.15
n_dir=8
n_sessions = 4

In [ ]:
for path in fish_list:
    print(path)
    traces = fl.load(path / "filtered_traces.h5", "/detr")   
    exp = LightsheetExperiment(path)
    fs = int(exp.fn)
    
    suite2p_brain = fl.load(path / "data_from_suite2p_cells_brain.h5")
    in_brain_idx = suite2p_brain['coords_idx']
    traces = traces[:, in_brain_idx]

    t = np.arange(np.shape(traces)[1]) / fs
    len_rec, num_traces = np.shape(traces)
    print("num_traces: ", num_traces)
    print("len_rec: ", len_rec)
    print("sampling rate: ", fs)
    
    try:
        regs = fl.load(path / "sensory_regressors.h5", "/regressors")[0]
        right = np.asarray(regs.iloc[:, 0])
        left = np.asarray(regs.iloc[:, 4])
    except:
        regs = fl.load(path / "sensory_regressors.h5", "/regressors")
        right = np.asarray(regs.iloc[:, 0])
        left = np.asarray(regs.iloc[:, 4])
        
    num_traces = np.shape(traces)[1]

    right_corr = np.zeros((num_traces))
    left_corr = np.zeros((num_traces))
    for i in range(num_traces):
        right_corr[i] = np.corrcoef(right, traces[:, i])[0,1]
        left_corr[i] = np.corrcoef(left, traces[:, i])[0,1]
        
    right_tuned = np.where(np.abs(right_corr) > thresh)[0]
    print(np.shape(right_tuned))
    n_right_tuned = np.shape(right_tuned)[0]

    left_tuned = np.where(np.abs(left_corr) > thresh)[0]
    print(np.shape(left_tuned))
    n_left_tuned = np.shape(left_tuned)[0]
    
    left_traces = traces[:, left_tuned].T
    right_traces = traces[:, right_tuned].T
    
    #### Getting a list of stimuli order: 0=right, 7=right-up
    pause_duration = int(exp['stimulus']['protocol']['E0040_motions_cardinal']['v31_8dir_plus_hd']['pause_duration']) * fs
    stim_duration = int(exp['stimulus']['protocol']['E0040_motions_cardinal']['v31_8dir_plus_hd']['moving_duration']) * fs

    left_diff = np.diff(left)
    right_diff = np.diff(right)

    left_start = find_peaks(left_diff, height=0.1)[0] - stim_duration - pause_duration
    left_end = find_peaks(left_diff, height=0.1)[0] + stim_duration + pause_duration
    right_start = find_peaks(right_diff, height=0.1)[0] - stim_duration - pause_duration
    right_end = find_peaks(right_diff, height=0.1)[0] + stim_duration + pause_duration
    
    num_left_trials = np.shape(left_start)[0]
    num_right_trials = np.shape(right_start)[0]
    len_segment = (pause_duration + stim_duration) * 2

    left_trials = np.zeros((n_dir, n_left_tuned, n_sessions, len_segment))
    right_trials = np.zeros((n_dir, n_right_tuned, n_sessions, len_segment))
    
    regs_array = np.asarray(regs)
    curr_session = np.zeros((n_dir), dtype=int)
    for i in range(num_left_trials):
        t1 = left_start[i]
        t2 = t1 + stim_duration

        curr_seg = np.nanmean(regs_array[t1:t2], axis=0)

        try:
            curr_dir = np.where(curr_seg > 0.1)[0][0]

            t1 = left_start[i]
            t2 = t1 + len_segment

            if curr_session[curr_dir] < n_sessions:
                left_trials[curr_dir, :, curr_session[curr_dir], :] = left_traces[:, t1:t2]
                curr_session[curr_dir] += 1
        except:
            print("Stupid trial")

    left_trials[left_trials == 0] = 'nan'
    
    curr_session = np.zeros((n_dir), dtype=int)
    for i in range(num_right_trials):
        t1 = right_start[i]
        t2 = t1 + stim_duration
        try:
            curr_seg = np.nanmean(regs_array[t1:t2], axis=0)
            curr_dir = np.where(curr_seg > 0.1)[0][0]

            t1 = right_start[i]
            t2 = t1 + len_segment


            if curr_session[curr_dir] < n_sessions:
                right_trials[curr_dir, :, curr_session[curr_dir], :] = right_traces[:, t1:t2]
                curr_session[curr_dir] += 1
        except:
            print("Stupid trial")

    right_trials[right_trials == 0] = 'nan'
    
    ####### Concatenate average responses and cluster
    left_trials_concat = np.zeros((n_left_tuned, len_segment * n_dir))

    for i in range(n_dir):
        lef_trials_avg = np.nanmean(left_trials[i], axis=1)
        t1 = i * len_segment
        t2 = t1 + len_segment
        left_trials_concat[:, t1:t2] = lef_trials_avg




    right_trials_concat = np.zeros((n_right_tuned, len_segment * n_dir))

    for i in range(n_dir):
        right_trials_avg = np.nanmean(right_trials[i], axis=1)
        t1 = i * len_segment
        t2 = t1 + len_segment
        right_trials_concat[:, t1:t2] = right_trials_avg

    d = {
        'concat_reordered_left_tuned_avg': left_trials_concat,
         'concat_reordered_right_tuned_avg': right_trials_concat,
        'reordered_trials_left_tuned': left_trials,
        'reordered_trials_right_tuned': right_trials,
    }
    fl.save(path / 'reordered_traces.h5', d)